In [3]:
from np_struct import Packet, Struct
import numpy as np

# Numpy Structures

`np-structures` extends structured arrays in NumPy to be a bit more user friendly and intuitive, with added support for transferring structured arrays across serial or socket interfaces. 
 
Structured arrays are built to mirror the struct typedef in C/C++, but can be used for any complicated data structure. They behave similar to standard arrays, but support mixed data types, labeling, and unequal length arrays. Arrays are easily written or loaded from disk in the standard `.npy` binary format.

## Creating Structs

The format for creating a new `Struct` type is similar to C/C++. Struct members are listed as class variables, and can be other Structs, numpy arrays, or one of the standard numpy types (e.g. np.uint8).

In [4]:
class pktheader(Struct):
    psize = np.uint16()
    dest =  np.uint8()
    src =   np.uint8()
    ptype = np.uint8()

When instancing a new struct object from the declared type, any member can be initialized by passing in it's value with a kwarg. The member will be broadcasted to match the shape of the initial value. 

In [5]:
pkt = pktheader(ptype=5)
pkt

Struct pktheader: 
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]

An array of structs can be created by passing in the `shape` kwarg. 

In [6]:
pkt_array = pktheader(ptype=5, shape=(3,2))
pkt_array

Struct pktheader: (3, 2)
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]
...
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]

Initialized members will be broadcasted to the shape of the initial value.

In [7]:
pkt = pktheader(ptype=np.arange(5))
pkt

Struct pktheader: 
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[0 1 2 3 4]

Structures can also include other structures.

In [8]:
class expkt(Struct):
    hdr = pktheader(ptype=0x2)
    da = np.arange(8)

ex = expkt()
ex

Struct expkt: 
    hdr:  Struct pktheader: 
            psize:  uint16[0]
            dest:   uint8[0]
            src:    uint8[0]
            ptype:  uint8[2]
    da:   int32[0 1 2 3 4 5 6 7]

## Accessing and Setting Members

Members can be accessed either with indexing, or using the dot operator.

In [9]:
ex.da *= 2
ex['da']

array([ 0,  2,  4,  6,  8, 10, 12, 14])

In [10]:
pkt_array[0,0].ptype = 15
pkt_array

Struct pktheader: (3, 2)
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[15]
]
...
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]

## Writing to Disk

In [11]:
np.save('test.npy', pkt_array)
pktheader(np.load('test.npy'))

Struct pktheader: (3, 2)
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[15]
]
...
[
    psize:  uint16[0]
    dest:   uint8[0]
    src:    uint8[0]
    ptype:  uint8[5]
]

## Interfaces

To send structured arrays across a serial or socket interface, the structure must include a member that holds the structure size in bytes, and another that holds a type field that is unique for each structure class that will be sent across the interface. 

To register the structure with the interface, the structure must inherit from a base class that includes the size and type fields, and implements four methods shown below.

In [13]:
from np_struct.fields import uint16

class BasePacket(Packet):
    hdr = pktheader()

    def set_size(self, value):
        self.hdr.psize = value

    def set_type(self, value):
        self.hdr.ptype = value

    def build_header(self, **kwargs):
        pass

    def parse_header(self, **kwargs):
        return dict(
            size = self.hdr.psize,
            type =  self.hdr.ptype,
        )

class command(Struct):
    state1 = uint16(bits=7)
    state2 = uint16(bits=3)
    state3 = uint16(bits=1)

class expkt(BasePacket):
    hdr = pktheader(ptype=0x2)
    cmd = command()
    da = np.arange(7)

class ack(BasePacket):
    hdr = pktheader(ptype=0xFF)
    ack_type = np.uint8()
    ack_code = np.uint8()

The `expkt` and `ack` structures will be recognized by an interface since they inherit from `BasePacket`, while `command` will not be recognized. However, `command` can be included inside other registered classes.

To implement an interface that recognizes these structures, pass the `BasePacket` class into the `pkt_class` kwarg.

In [14]:
from np_struct.transfer import SocketInterface
import threading

server = SocketInterface(host=('localhost', 50007), pkt_class=BasePacket)

client = SocketInterface(target=('localhost', 50007), pkt_class=BasePacket)

def start_server():
    with server as intf:
        pkt = server.pkt_read()
        pkt.hdr.src = 0xF
        pkt.da *= 2
        pkt.cmd.state2 = 0x3
        intf.pkt_write(pkt)

threading.Thread(target=start_server).start()

with client as intf:

    ex = expkt()
    print(ex)
    print()
    rxpkt = intf.pkt_sendrecv(ex)

    print(rxpkt)

BasePacket expkt: 
    hdr:  Struct pktheader: 
            psize:  uint16[35]
            dest:   uint8[0]
            src:    uint8[0]
            ptype:  uint8[2]
    cmd:  Struct command: 
            state1:       uint16(7:0)[0]
            state2:       uint16(10:7)[0]
            state3:       uint16(11:10)[0]
    da:   int32[0 1 2 3 4 5 6]

BasePacket expkt: 
    hdr:  Struct pktheader: 
            psize:  uint16[35]
            dest:   uint8[0]
            src:    uint8[15]
            ptype:  uint8[2]
    cmd:  Struct command: 
            state1:       uint16(7:0)[0]
            state2:       uint16(10:7)[3]
            state3:       uint16(11:10)[0]
    da:   int32[ 0  2  4  6  8 10 12]
